In [2]:
from web3 import Web3, Account

In [3]:
local_ganache_http_provider = 'HTTP://127.0.0.1:7545'

In [4]:
w3=Web3(Web3.HTTPProvider(local_ganache_http_provider))

In [6]:
w3.eth.get_block('latest')

AttributeDict({'number': 0,
 'hash': HexBytes('0x65cdf72ef721839715d2f80cf1ad84694d4eb4060a734eb5c81acd6d14be7b29'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [12]:
accounts = w3.eth.accounts

In [23]:
"""
Sample Transaction
    {
        transaction_id: int,
        customer_id: address_from_ganache,
        transaction_amount: int(in dollars),
        transaction_category: ['gas', 'restaurant',  'grocery', 'subscription', 'entertainment'],
        transaction_date: Date
    }
"""

"\nSample Transaction\n    {\n        transaction_id: int,\n        customer_id: address_from_ganache,\n        transaction_amount: int(in dollars),\n        transaction_category: ['gas', 'restaurant',  'grocery', 'subscription', 'entertainment'],\n        transaction_date: Date\n    }\n"

In [27]:
from random import choice
import random
from datetime import datetime
from time import sleep
txn_categories = ['gas', 'restaurant',  'grocery', 'subscription', 'entertainment']

In [28]:
for i in range(50):
    tx = {}
    tx['id'] = i
    tx['customer_id'] = choice(accounts)
    tx['amount'] = round(random.uniform(0.5, 99.5), 2)
    tx['category'] = choice(txn_categories)
    tx['date'] = datetime.now()
    sleep(1)
    print(tx)

{'id': 0, 'customer_id': '0x613c66B0D1cF9aF2cdf5c0D790A44edFa6Fc25B1', 'amount': 28.19, 'category': 'grocery', 'date': datetime.datetime(2022, 6, 1, 22, 24, 8, 346131)}
{'id': 1, 'customer_id': '0xEb22c8DA84af3f53030367A1c0fC0a356a70c5A9', 'amount': 98.19, 'category': 'subscription', 'date': datetime.datetime(2022, 6, 1, 22, 24, 9, 351024)}
{'id': 2, 'customer_id': '0x5e0d71269A0CEf963f15e198D073031DE84260ca', 'amount': 33.49, 'category': 'entertainment', 'date': datetime.datetime(2022, 6, 1, 22, 24, 10, 356141)}
{'id': 3, 'customer_id': '0x5e0d71269A0CEf963f15e198D073031DE84260ca', 'amount': 42.25, 'category': 'entertainment', 'date': datetime.datetime(2022, 6, 1, 22, 24, 11, 361055)}
{'id': 4, 'customer_id': '0xEb22c8DA84af3f53030367A1c0fC0a356a70c5A9', 'amount': 80.49, 'category': 'grocery', 'date': datetime.datetime(2022, 6, 1, 22, 24, 12, 366088)}
{'id': 5, 'customer_id': '0x5e0d71269A0CEf963f15e198D073031DE84260ca', 'amount': 95.55, 'category': 'grocery', 'date': datetime.datetim